In [2]:
import sys
sys.path.append('/home/lessa/smodels')
from smodels.experiment.databaseObj import Database
from smodels.tools.physicsUnits import GeV,fb,pb
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

In [3]:
db = Database('/home/lessa/smodels-database')

In [4]:
exp = db.getExpResults(analysisIDs=['ATLAS-CONF-2013-065'],dataTypes=['upperLimit'],txnames=['T6bbWWoff'],
                       useSuperseded=True,useNonValidated=True)[0]

In [5]:
tx = exp.getTxNames()[0]
print tx

T6bbWWoff


In [7]:
data = tx.txnameData._data

In [8]:
xvals = [pt[0][0][0].asNumber(GeV) for pt in data]
yvals = [pt[0][0][-1].asNumber(GeV) for pt in data]
zvals = [pt[1].asNumber(pb) for pt in data]

In [25]:
import random
xvals = [random.uniform(100.,150.) for i in range(100)]
yvals = [xvals[i]-random.uniform(0.,81.) for i in range(100)]
zvals = [tx.txnameData.getValueFor([[300.*GeV,xvals[i]*GeV,yvals[i]*GeV]]*2) for i in range(100)]
for i,pt in enumerate(zvals):
    if pt is None:
        zvals[i] = -300.
    else:
        zvals[i] = pt.asNumber(pb)

In [26]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xvals, yvals, zvals)
ax.set_xlabel('M1')
ax.set_ylabel('m')
ax.set_zlabel('UL')

plt.show()

In [6]:
m = [[1358.10371939*GeV,638.216496528*GeV],[1414.69137436*GeV,638.216496528*GeV]]
m[0][0] = m[1][0]*0.96
porig = tx.txnameData.flattenMassArray(m)
pt = ( (np.matrix(porig)[0] - tx.txnameData.delta_x ) ).tolist()[0]
pt = np.dot(pt,tx.txnameData._V)  ## rotate
pt = np.array(pt[:tx.txnameData.dimensionality])
print 'Pt=',pt,'val=',tx.txnameData.getValueFor(m)
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ptm = [pt[0],pt[1],tx.txnameData.getValueFor(m).asNumber(pb)]
# ax.scatter([ptm[0]], [ptm[1]], [ptm[2]],c='r',marker='o')
xpts = []
ypts = []
zpts = []
for ipt in tx.txnameData.tri.simplices[tx.txnameData.tri.find_simplex(pt)]:
    xpts.append(tx.txnameData.tri.points[ipt][0])
    ypts.append(tx.txnameData.tri.points[ipt][1])
    if tx.txnameData.dimensionality == 3:
        zpts.append(tx.txnameData.tri.points[ipt][2])
    else:
        zpts.append(tx.txnameData.xsec[ipt])
    print tx.txnameData.tri.points[ipt],tx.txnameData.xsec[ipt]

Pt= [  3.08248334e+02  -6.79790160e+01  -5.71193013e-03] val= 9.34E-03 [pb]
[  4.80880887e+02  -5.69078593e+02   8.77068655e-03] 2.16E-01 
[  3.43573361e+02  -5.12460194e+02   9.23385053e-03] 1.08E-01 
[  2.89711308e+02  -6.50872185e+02   9.65360063e-03] 2.60E-01 
[  3.17570452e+02  -5.82038198e+02  -4.98716741e-03] 1.95E-01 


In [ ]:
print tx.txnameData.countNonZeros(pt),tx.txnameData.dimensionality

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xpts, ypts, zpts,s=80,c='r',marker='o')
ax.scatter([pt[0]],[pt[1]],[pt[2]],s=80,c='b',marker='o')
ax.set_xlabel('x')
ax.set_ylabel('y')
if tx.txnameData.dimensionality == 3:
    ax.set_zlabel('UL')
else:
    ax.set_zlabel('z')

plt.show()

In [ ]:
uvw = [pt[:3]]
fill_value=np.nan
tol = sys.float_info.epsilon * 1e06
simplex = tx.txnameData.tri.find_simplex(uvw, tol=tol)
vertices = np.take(tx.txnameData.tri.simplices, simplex, axis=0)
temp = np.take(tx.txnameData.tri.transform, simplex, axis=0)
d=temp.shape[2]
delta = uvw - temp[:, d]
bary = np.einsum('njk,nk->nj', temp[:, :d, :], delta)
tx.txnameData.vtx = vertices
tx.txnameData.wts = np.hstack((bary, 1 - bary.sum(axis=1, keepdims=True)))
v=tx.txnameData.xsec
if type(tx.txnameData.xsec[0]) == float:
    values = np.array ( [ float(x) for x in tx.txnameData.xsec ] )
else:
    values = np.array ( [ x.asNumber() for x in tx.txnameData.xsec ] )
print 'v=',tx.txnameData.wts
ret = np.einsum('nj,nj->n', np.take(values, tx.txnameData.vtx), tx.txnameData.wts)
print 'r=',ret
with np.errstate(invalid='ignore'):
    ret[np.any(tx.txnameData.wts < -1e-10, axis=1)] = fill_value
print ret

In [ ]:
fig.clear()
plt.triplot(xpts,ypts)
plt.plot([ptm[0]], [ptm[1]],'ro',xpts,ypts,'bo')
plt.show()

In [ ]:
tri = tx.txnameData.tri
simplex = tri.find_simplex([pt])
vertices = np.take(tri.simplices, simplex, axis=0)
temp = np.take(tri.transform, simplex, axis=0)
d=temp.shape[2]
delta = [pt] - temp[:, d]
bary = np.einsum('njk,nk->nj', temp[:, :d, :], delta)
vtx = vertices
wts = np.hstack((bary, 1 - bary.sum(axis=1, keepdims=True)))
v=tx.txnameData.xsec
values = np.array ( [ x.asNumber() for x in tx.txnameData.xsec ] )
ret = np.einsum('nj,nj->n', np.take(values, vtx), wts)
with np.errstate(invalid='ignore'):
    ret[np.any(wts < -1e-10, axis=1)] = np.nan
print ret[0]

In [ ]:
vertices

In [ ]:
pt

In [ ]:
m = [[347.406576078*GeV,166.767198233*GeV],[347.406576078*GeV,166.767198233*GeV]]
porig = tx.txnameData.flattenMassArray(m)
ptA = ( (np.matrix(porig)[0] - tx.txnameData.delta_x ) ).tolist()[0]
ptA = np.dot(ptA,tx.txnameData._V)  ## rotate
print ptA,tx.txnameData.countNonZeros(ptA)

In [ ]:
print tx.txnameData.getValueFor(m)